# Basic RAG over an existing vector database

In [1]:
#!pip install langchain langchain_ollama
#!pip install chromadb sentence-transformers langchain_huggingface langchain_chroma

Select **model** for embeddings. 

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/bigdata/miniconda3/envs/Trabajo_BIDA/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Access existing database

In [3]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="some_facts",
    embedding_function=embeddings,
    persist_directory="./chroma_some_facts",  # Where to save data locally, remove if not necessary
)

Create retriever

In [4]:
retriever = vector_store.as_retriever()

Make a search in the vectorstor (only for testing purposes)

In [5]:
question = "Can I steal a bank?"
docs = vector_store.similarity_search(question)
len(docs)

4

In [6]:
docs[0]

Document(metadata={}, page_content='Robbers broke into the city bank and stole $1 million in cash.')

RAG Chain

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [8]:
# Question
chain.invoke("Can I steal a bank?")

'No, you cannot steal a bank. The context provided is about robbers breaking into a city bank and stealing money, not about physically stealing or taking over a bank as an entity.'